In [42]:
import numpy as np
import os
import pandas as pd
import snowflake.connector
from PIL import Image
from pytesseract import pytesseract
import glob 
import datetime as dt



In [29]:
%%time
def sf_conn(user, pwd, url, wh=''):
    cnx = snowflake.connector.connect(
        user=user,
        password=pwd,
        account=url,
        warehouse=wh
    )
    return cnx.cursor()


def exec_query(query, user, pwd, url):
    cur = sf_conn(user, pwd, url)
    get_data = cur.execute(query).fetchall()
    column_names = [i[0] for i in cur.description]
    cur.close()
    return (pd.DataFrame(get_data, columns=column_names))

#-o datasette.png

query = ("""select * from ;""")
df = exec_query(query, '', '', '')

ctr=0
for index, row in df.iterrows():
    try:
        filename=str(row['Ticker'])+'-'+str(row['CIK'])+".png"
        s_url = 'shot-scraper https://pcaobus.org/resources/auditorsearch/issuers/?issuercik='+str(row['CIK']) + ' -o ' + filename + ' --wait=2500'
        os.system(s_url)
        if ctr % 487 == 0:
            print('=> ', ctr,)
    except Exception as e:
        print('😫', ctr, filename, e)
        ctr=ctr+1
        continue
    ctr=ctr+1

=>  0
=>  487
=>  974
=>  1461
=>  1948
=>  2435
=>  2922
=>  3409
=>  3896
=>  4383
=>  4870
=>  5357
=>  5844
=>  6331
=>  6818
=>  7305
=>  7792
=>  8279
=>  8766
=>  9253
=>  9740
=>  10227
=>  10714
=>  11201
=>  11688
CPU times: user 15.7 s, sys: 7.65 s, total: 23.4 s
Wall time: 18h 36min 42s


In [11]:
df.shape[0]

12175

In [43]:
%%time

path_to_tesseract = "/usr/local/Cellar/tesseract/5.2.0/bin/tesseract" #r"C:\Program Files\Tesseract-OCR\tesseract.exe"
split_list = ['YEAR', 'ENGAGEMENT PARTNER', 'FIRM' , 'ISSUER']

error_list = [] 
df3 = pd.DataFrame(columns=['FORM_AP_FILING', 'YEAR', 'ENGAGEMENT_PARTNER', 'FIRM', 'ISSUER',  'CIK'], dtype=object)

ctr=0
for file in glob.glob("*.png"):
    try:
        formList=[]
        yearList=[]
        engagement_list = []
        firm_list = []
        issuer_list = []
        cik_list=[]
        ticker_list=[]

    # Opening the image & storing it in an image object
        img = Image.open(file)
  
    # Providing the tesseract executable
    # location to pytesseract library
        pytesseract.tesseract_cmd = path_to_tesseract
  
    # Passing the image object to image_to_string() function
    # This function will extract the text from the image
        text = pytesseract.image_to_string(img)
    
        counter = 0
        for i in text.split('FORM AP FILING'):
            if (counter == 0 or len(i) < 3):
                counter=counter+1
                continue
            formList.append(i[0:15].replace('\n', ''))
            
            
            cik_list.append(file.split('.png')[0].split('-')[-1])
        
            for j in split_list:
                if j =='YEAR':
                    yearList.append(i.split(j)[1][:6].replace('\n', ''))
                elif j == 'ENGAGEMENT PARTNER':
                    engagement_list.append(i.split(j)[1].split('FIRM')[0].replace('\n', ' '))
                elif j == 'FIRM':
                    firm_list.append(i.split(j)[1].split('ISSUER')[0].replace('\n', ' '))
                elif j == 'ISSUER':
                    issuer_list.append(i.split(j)[1].replace('\n', ' '))
                

        
        df2 = pd.DataFrame( {'FORM_AP_FILING': formList, 'YEAR': yearList, 'ENGAGEMENT_PARTNER': engagement_list,
                    'FIRM': firm_list,'ISSUER': issuer_list, 'CIK': cik_list })
        df3=df3.append(df2)
    except Exception as e:
        ctr=ctr+1
        error_list.append(str(file)+  ' 😫 ' + str(e))
        continue
    
    if ctr % 487 == 0:
        print('=> ', ctr, df3.shape[0], dt.datetime.now())
    ctr=ctr+1
        
        
df3.to_csv('data2.csv', index=False)
        
#         print(file, e)

=>  0 2 2022-09-30 19:59:07.756248
=>  487 1713 2022-09-30 20:19:07.085264
=>  974 3472 2022-09-30 20:38:19.229157
=>  1461 5238 2022-09-30 20:57:47.453990
=>  2435 8675 2022-09-30 21:36:10.781210
=>  2922 10403 2022-09-30 21:55:11.809441
=>  3409 12016 2022-09-30 22:15:18.963824
=>  3896 13720 2022-09-30 22:34:18.784083
=>  4383 15395 2022-09-30 22:53:06.051240
=>  4870 17171 2022-09-30 23:12:14.269032
=>  5357 18877 2022-09-30 23:31:32.239573
=>  6331 22245 2022-10-01 00:09:29.174302
=>  6818 23864 2022-10-01 00:28:28.836195
=>  7792 27181 2022-10-01 01:07:06.666254
=>  8279 28938 2022-10-01 01:25:51.603474
=>  8766 30600 2022-10-01 01:44:43.869219
=>  9253 32375 2022-10-01 02:03:32.742135
=>  9740 34082 2022-10-01 02:22:23.939927
=>  11201 39183 2022-10-01 03:18:22.087548
CPU times: user 1h 19min 18s, sys: 3min 56s, total: 1h 23min 15s
Wall time: 7h 56min 25s


In [38]:
dfe = pd.DataFrame( {'ERRORS': error_list })

In [44]:
df3.shape

(42600, 6)

In [45]:
dfe.shape

(1020, 1)

In [46]:
dfe.head()

,ERRORS
0,ob-1454938.png 😫 list index out of range
1,mgrb-1004434.png 😫 list index out of range
2,pht-1166258.png 😫 list index out of range
3,tlfa-909724.png 😫 list index out of range
4,vrtc-773318.png 😫 list index out of range


In [80]:
df3=pd.read_csv('data2.csv')

In [81]:
import datetime as dt
for index, row in df3.iterrows():
    if len(str(row['ENGAGEMENT_PARTNER']).strip()) < 2:
        df3.at[index, 'ENGAGEMENT_PARTNER'] = ''
    try:
        s = str(row['FORM_AP_FILING']).replace('.', '').strip()
        d = dt.datetime.strptime(s, '%b %d, %Y')
        df3.at[index, 'FORM_AP_FILING'] = str(d)[:10]
        
    except Exception as e:
        continue

In [82]:
df3 = df3.sort_values(by=['CIK', 'FORM_AP_FILING', 'YEAR'], ascending=False)
df3.head()

,FORM_AP_FILING,YEAR,ENGAGEMENT_PARTNER,FIRM,ISSUER,CIK
9968,2022-07-27,,,,,1916902
13987,2022-09-23,2021,Benny Scott Hickson,Ernst & Young LLP (42),"HF Sinclair Corporation (DINO | 0001915657) Sort By _ Filing Date (Newest) ~ € 1 ofl > 24 Items Per Page About Submit a Tip . Subscribe to our Newsletter Oversight Comment Now * indicates required Resources AuditorSearch Email address * lidentify myself as : oe What type of updates would you like to receive? * Contact the Firm Filings oe G u CAI PCAOB Updates _[] Inspection Reports PCAOB Communications oO Enforcement to Audit Orders PCAOB Careers Committee Members News Releases, O Speeches, and Communications Statements to Auditors Communications to Investors",1915657
41802,2022-09-07,,,,,1915328
38933,2022-03-14,2021,Joseph Adam Jalbert,"Baker Newman & Noyes, P.A. Limited Liability Company (231)","ECB Bancorp, Inc. (ECBK | 0001914605) Sort By _ Filing Date (Newest) ~ € 1 ofl > 24 Items Per Page About Submit a Tip . Subscribe to our Newsletter Oversight Comment Now * indicates required Resources AuditorSearch Email address * lidentify myself as oe What type of updates would you like to receive? * Contact the Firm Filings CAI PCAOB Updates _[] Inspection Reports PCAOB Communications oO Enforcement to Audit Orders PCAOB Careers Committee Members News Releases, O Speeches, and Communications Statements to Auditors Communications to Investors",1914605
25706,2022-04-18,,,,,1913577


In [84]:
df3.loc[df3['ENGAGEMENT_PARTNER'].notnull()].shape

(42600, 6)

In [85]:
df3.shape

(42600, 6)

In [86]:
df3.to_csv('sort_Data.csv', index=False)